# Книжная база SQL



In [1]:
import pandas as pd
from sqlalchemy import create_engine

## Общие данные

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### Просмотр таблицы books 

In [3]:
query = ''' 
SELECT * 
FROM books 
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Просмотр таблицы  authors

In [4]:
query = ''' 
SELECT * 
FROM authors 
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Просмотр таблицы  publishers

In [5]:
query = ''' 
SELECT * 
FROM publishers 
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)



,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Просмотр таблицы  ratings

In [6]:
query = ''' 
SELECT * 
FROM ratings 
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)



,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Просмотр таблицы  reviews

In [7]:
query = ''' 
SELECT * 
FROM reviews 
LIMIT 5'''
pd.io.sql.read_sql(query, con = engine)



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [8]:
query = ''' 
SELECT 
COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


Книг получается 819

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query = ''' 
SELECT 
b.book_id,
COUNT(r.review_id) AS coutn_of_reviews,
AVG(ra.rating) AS AVG_rating
FROM books AS b
LEFT JOIN reviews  AS r ON b.book_id=r.book_id
LEFT JOIN  ratings AS ra ON b.book_id=ra.book_id
GROUP BY b.book_id
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,coutn_of_reviews,avg_rating
0,652,4,4.500000
1,273,4,4.500000
2,51,60,4.250000
3,951,4,4.000000
4,839,28,4.285714


### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = ''' 
SELECT 
publisher,
COUNT(b.book_id)
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY COUNT(b.book_id) DESC 
LIMIT 1'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = ''' 
SELECT 
a.author,
AVG(r.rating)
FROM books AS b
LEFT JOIN authors AS a ON b.author_id=a.author_id
LEFT JOIN ratings AS r ON b.book_id=r.book_id
GROUP BY a.author_id, b.book_id
HAVING COUNT(r.username) >= 50
ORDER BY AVG(r.rating) DESC 
LIMIT 1'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.414634


### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query = ''' 
WITH 
a AS
(SELECT 
ra.username
FROM ratings AS ra
GROUP BY ra.username
HAVING count(ra.rating_id) > 50),
b AS (SELECT
r.username,
COUNT(r.review_id) AS count_reviews
FROM reviews AS r
GROUP BY r.username)
SELECT AVG(b.count_reviews)
FROM b 
JOIN a ON b.username=a.username '''
pd.io.sql.read_sql(query, con = engine)
  

,avg
0,24.333333


В среднем, пользователь, поставивший больше 50-ти оценок составляет 24 обзора на книги

Выводы:

В ходе анализа базы данных, содержащей информацию о книгах (в том числе об авторах, издательствах, оценках и обзорах), были определены следующие факты:

819 книг было изданы после 1 января 2000

Издательством, которое выпустило больше всего книг (брошюры не учитывались), является Penguin Books

Автором с самыми высокими оценками (в разрезе книг автора или всех книг из БД) является J.K. Rowling

Пользователи, оставляющие более 50 оценок к книгам, в среднем делают по 24 обзора

